In [44]:
from pymongo import MongoClient

In [45]:
client = MongoClient("mongodb://mongodb:27017/")

In [46]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.config("spark.mongodb.input.partitioner", "MongoPaginateByCountPartitioner") \
.config("spark.mongodb.input.partitionerOptions.MongoPaginateByCountPartitioner.numberOfPartitions", "10") \
.getOrCreate()

In [109]:
influenza = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.influenza").load()

In [8]:
#influenza.show()

In [5]:
influenza.count()

91998

In [110]:
influenza = influenza.selectExpr("_id", "country", "datefs", "stack(12, 'AH1', type_A_H1, 'AH1N12009', type_A_H1N1, 'AH3', type_A_H3, 'AH5', type_A_H5, \
'ANOTSUBTYPED', type_A_NS, 'BYAMAGATA', type_B_Yamata, 'BVICTORIA', type_B_Victoria, 'BNOTDETERMINED', type_B_ND) as (souche, cases)")

In [111]:
influenza.show()

+--------------------+-------+----------+--------------+-----+
|                 _id|country|    datefs|        souche|cases|
+--------------------+-------+----------+--------------+-----+
|[5ec2c7322990bce6...|Algeria|2010-01-10|           AH1|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|     AH1N12009|   37|
|[5ec2c7322990bce6...|Algeria|2010-01-10|           AH3|   23|
|[5ec2c7322990bce6...|Algeria|2010-01-10|           AH5|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|  ANOTSUBTYPED|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|     BYAMAGATA|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|     BVICTORIA|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|BNOTDETERMINED|    0|
|[5ec2c7322990bce6...|Algeria|2010-01-10|          null| null|
|[5ec2c7322990bce6...|Algeria|2010-01-10|          null| null|
|[5ec2c7322990bce6...|Algeria|2010-01-10|          null| null|
|[5ec2c7322990bce6...|Algeria|2010-01-10|          null| null|
|[5ec2c7322990bce6...|Algeria|2010-01-17|           AH1

In [112]:
influenza.createOrReplaceTempView("influenza")

In [113]:
influenza = spark.sql("SELECT _id, country, datefs AS date, CASE WHEN souche like 'A%' THEN 'A' WHEN souche like 'B%' THEN 'B' END as type, souche, cases, \
COALESCE(int((sum(cases) over (partition by country, souche ORDER BY datefs asc))), 0) as cumul_cases FROM influenza")
influenza.show()

+--------------------+-------+----------+----+------+-----+-----------+
|                 _id|country|      date|type|souche|cases|cumul_cases|
+--------------------+-------+----------+----+------+-----+-----------+
|[5ec2c7332990bce6...| Belize|2010-01-10|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-01-17|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-01-24|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-01-31|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-02-07|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-02-14|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-02-21|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-02-28|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-03-07|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-03-14|   A|   AH3|    0|          0|
|[5ec2c7332990bce6...| Belize|2010-03-21|   A|   AH3|    0|     

In [102]:
#influenza.createOrReplaceTempView("influenza")

In [104]:
#influenza = spark.sql("SELECT _id, country, datefs AS date, type, cases, \
#COALESCE(int((sum(cases) over (partition by country, type ORDER BY datefs asc))), 0) as cumul_cases FROM influenza")
#influenza.show()

In [10]:
#influenza = spark.sql("SELECT _id, country, datefs AS date,\
#type_A_H1 AS AH1, type_A_H1N1 AS AH1N12009, type_A_H3 AS AH3, type_A_H5 AS AH5, type_A_NS ANOTSUBTYPED,\
#type_A_total AS INF_A, type_B_Yamata AS BYAMAGATA, type_B_Victoria AS BVICTORIA, type_B_ND AS BNOTDETERMINED,\
#type_B_total AS INF_B, total_positives AS ALL_INF, total_negatives AS ALL_INF2 FROM influenza")
#influenza.show()

In [ ]:
#influenza = spark.sql("SELECT _id, country, datefs AS date,\
#type_A_H1 AS AH1, type_A_H1N1 AS AH1N12009, type_A_H3 AS AH3, type_A_H5 AS AH5, type_A_NS ANOTSUBTYPED,\
#type_A_total AS INF_A, type_B_Yamata AS BYAMAGATA, type_B_Victoria AS BVICTORIA, type_B_ND AS BNOTDETERMINED,\
#type_B_total AS INF_B, total_positives AS ALL_INF, total_negatives AS ALL_INF2 FROM influenza")

In [28]:
#days_week = spark.sql("SELECT distinct (year, week) FROM influenza")

In [29]:
#days_week = spark.sql("SELECT year, week, dateds AS week_start_date, datefs AS week_end_date FROM influenza GROUP BY year, week, week_start_date, week_end_date")

In [20]:
#days_week.show()

In [114]:
influenza.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_influenza").save()

In [12]:
#days_week.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics.days_week").save()